<a href="https://colab.research.google.com/github/lin941006/Time/blob/main/%E5%80%8B%E4%BA%BA%E6%99%82%E9%96%93%E8%A6%8F%E5%8A%83_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio pandas
import gradio as gr
import pandas as pd
from datetime import datetime, timedelta

# ---------------------------
# 儲存代辦事項
# ---------------------------
tasks = []

# ---------------------------
# 分類與細項
# ---------------------------
category_choices = [
    "生活類",
    "工作/學習類",
    "通勤類",
    "休閒娛樂類",
    "社交互動類",
    "運動與健康類",
    "家務/生活管理類",
    "其他"
]

subcategory_choices = {
    "生活類": ["睡覺", "起床", "吃飯", "洗澡", "休息/發呆"],
    "工作/學習類": ["上班", "上課", "寫作業", "自習", "開會", "報告", "考試", "線上課程"],
    "通勤類": ["捷運", "公車", "火車", "走路", "騎車", "開車"],
    "休閒娛樂類": ["玩手機", "看電影", "追劇", "聽音樂", "看書", "逛街", "看展覽"],
    "社交互動類": ["聊天", "打電話", "聚會", "約會", "家庭時間", "同事交流"],
    "運動與健康類": ["健身", "跑步", "散步", "拉筋", "瑜珈", "冥想", "看醫生"],
    "家務/生活管理類": ["打掃", "洗衣服", "曬衣服", "煮飯", "採買食材", "整理房間", "倒垃圾"]
}

# ---------------------------
# 一天開始時間（24 小時制，每 30 分鐘）
# ---------------------------
time_choices = [f"{str(h).zfill(2)}:{m}" for h in range(24) for m in ["00", "30"]]

# 預計花費時間（0.5 ~ 8 小時，每 0.5 小時）
duration_choices = [i * 0.5 for i in range(1, 17)]

# ---------------------------
# 控制細項顯示
# ---------------------------
def toggle_subcategory(category, _):
    if category == "其他":
        # 選擇其他分類，只顯示文字輸入框
        return gr.update(visible=False, value=None), gr.update(visible=True, value="")
    elif category:
        sub_choices = subcategory_choices.get(category, [])
        return gr.update(choices=sub_choices, visible=True, value=None), gr.update(visible=False, value="")
    else:
        return gr.update(visible=False, value=None), gr.update(visible=False, value="")

# ---------------------------
# 新增代辦事項
# ---------------------------
def add_task(category, subcategory, other_input, hours):
    if not category or hours is None:
        return pd.DataFrame(tasks), None, None, "", None

    if category == "其他":
        task_name = other_input.strip()
        if not task_name:
            return pd.DataFrame(tasks), None, None, "", None
    else:
        if not subcategory:
            return pd.DataFrame(tasks), None, None, "", None
        task_name = subcategory

    tasks.append({
        "事項": task_name,
        "所需時間(小時)": hours
    })

    # 清空輸入欄位
    return pd.DataFrame(tasks), None, None, "", None

# ---------------------------
# 產生行程
# ---------------------------
def generate_schedule(start_time):
    if not tasks or not start_time:
        return "請先新增代辦事項並選擇一天開始時間。"

    current_time = datetime.strptime(start_time, "%H:%M")
    schedule_text = "📅 今日行程安排：\n\n"

    for task in tasks:
        end = current_time + timedelta(hours=task["所需時間(小時)"])
        schedule_text += f"{current_time.strftime('%H:%M')} ～ {end.strftime('%H:%M')}：{task['事項']}\n"
        current_time = end

    schedule_text += "\n🎉 祝福你有美好的一天！要開心喲~"
    return schedule_text

# ---------------------------
# 清除全部資料
# ---------------------------
def clear_all():
    tasks.clear()
    # 六個輸出對應 task_table, category, subcategory, other_input, time_input, start_time, schedule_output
    return pd.DataFrame(tasks), None, None, "", None, None, ""

# ---------------------------
# Gradio 介面
# ---------------------------
with gr.Blocks(title="個人時間管理工具") as demo:
    gr.Markdown("## 🕒 個人時間管理工具")
    gr.Markdown("依照分類選擇代辦事項，選擇預計花費時間，系統會自動安排行程並在畫面呈現。")

    with gr.Row():
        category_input = gr.Dropdown(
            choices=category_choices,
            value=None,
            label="代辦事項分類"
        )

        subcategory_input = gr.Dropdown(
            choices=[],
            value=None,
            label="細項代辦事項",
            visible=False
        )

        other_input = gr.Textbox(
            label="其他事項",
            visible=False
        )

        time_input = gr.Dropdown(
            choices=duration_choices,
            value=None,
            label="預計花費時間（小時）"
        )

    category_input.change(
        toggle_subcategory,
        inputs=[category_input, subcategory_input],
        outputs=[subcategory_input, other_input]
    )

    add_btn = gr.Button("➕ 新增今日代辦事項")
    task_table = gr.Dataframe(label="📋 今日代辦清單", interactive=False)

    add_btn.click(
        add_task,
        inputs=[category_input, subcategory_input, other_input, time_input],
        outputs=[task_table, category_input, subcategory_input, other_input, time_input]
    )

    gr.Markdown("---")

    start_time = gr.Dropdown(
        choices=time_choices,
        value=None,
        label="開始時間"
    )

    schedule_btn = gr.Button("📅 產生今日行程")
    schedule_output = gr.Textbox(lines=12, label="系統安排的行程")

    # 清除按鈕（右下角）
    clear_btn = gr.Button("🗑️ 清除所有行程")

    schedule_btn.click(
        generate_schedule,
        inputs=start_time,
        outputs=schedule_output
    )

    clear_btn.click(
        clear_all,
        outputs=[task_table, category_input, subcategory_input, other_input, time_input, start_time, schedule_output]
    )

demo.launch()


In [1]:
print('s')

s
